PART A (Data preparetion)

In [ ]:

# Cell 1: Imports & Settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import sqlite3
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [4]:
# Cell 2: Part A – Load & Document
fear_greed = pd.read_csv('fear_greed_index.csv')
trader = pd.read_csv('historical_data.csv', low_memory=False)   # huge file → low_memory

print("=== Fear & Greed ===")
print("Rows:", len(fear_greed), "Columns:", fear_greed.shape[1])
print(fear_greed.isnull().sum())
print("Duplicates:", fear_greed.duplicated().sum())

print("\n=== Trader Data ===")
print("Rows:", len(trader), "Columns:", trader.shape[1])
print(trader.isnull().sum()[trader.isnull().sum() > 0])
print("Duplicates:", trader.duplicated().sum())

=== Fear & Greed ===
Rows: 2644 Columns: 4
timestamp         0
value             0
classification    0
date              0
dtype: int64
Duplicates: 0

=== Trader Data ===
Rows: 211224 Columns: 16
Series([], dtype: int64)
Duplicates: 0


In [9]:
df['Timestamp IST'].head(5)

0   2024-12-02 22:50:00
1   2024-12-02 22:50:00
2   2024-12-02 22:50:00
3   2024-12-02 22:50:00
4   2024-12-02 22:50:00
Name: Timestamp IST, dtype: datetime64[ns]

In [5]:
# Cell 3: Convert timestamps & align by DATE (daily level)
fear_greed['date'] = pd.to_datetime(fear_greed['date']).dt.date

trader['Timestamp IST'] = pd.to_datetime(trader['Timestamp IST'], format='%d-%m-%Y %H:%M', errors='coerce')
trader['trade_date'] = trader['Timestamp IST'].dt.date

# Merge on date
df = trader.merge(fear_greed[['date', 'classification']], 
                  left_on='trade_date', 
                  right_on='date', 
                  how='left')
df['sentiment'] = df['classification'].fillna('Neutral')  # very few missing
print("Merged shape:", df.shape)

Merged shape: (211224, 20)


In [14]:
# ==================== CELL 4: FIXED Key Metrics (Part A-3) ====================
# FIXED: Removed 'leverage' (column not present) + safer calculations

daily = df.groupby(['Account', 'trade_date', 'sentiment']).agg(
    total_pnl=('Closed PnL', 'sum'),
    num_trades=('Trade ID', 'nunique'),
    avg_size_usd=('Size USD', 'mean'),
    # Fixed long_ratio using both Side and Direction columns
    long_ratio=('Side', lambda x: (
        (x.str.contains('BUY|Long', case=False, na=False)) | 
        (df.loc[x.index, 'Direction'].str.contains('Long|Buy', case=False, na=False))
    ).mean())
).reset_index()

# Win rate calculation (only trades with PnL)
win_rate = df[df['Closed PnL'] != 0].groupby(['Account', 'trade_date'])['Closed PnL'].apply(
    lambda x: (x > 0).mean()
).reset_index(name='win_rate')

daily = daily.merge(win_rate, on=['Account', 'trade_date'], how='left')
daily['win_rate'] = daily['win_rate'].fillna(0)

print("✅ Daily metrics created successfully!")
print(daily.head())
print("\nColumns in daily:", list(daily.columns))

✅ Daily metrics created successfully!
                                      Account  trade_date      sentiment  \
0  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-11  Extreme Greed   
1  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-17  Extreme Greed   
2  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-18  Extreme Greed   
3  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-22  Extreme Greed   
4  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-26  Extreme Greed   

   total_pnl  num_trades  avg_size_usd  long_ratio  win_rate  
0        0.0         155   5089.718249    0.000000       0.0  
1        0.0          65   7976.664412    0.000000       0.0  
2        0.0          40  23734.500000    0.000000       0.0  
3   -21227.0          12  28186.666667    1.000000       0.0  
4     1603.1          27  17248.148148    0.444444       1.0  

Columns in daily: ['Account', 'trade_date', 'sentiment', 'total_pnl', 'num_trades', 'avg_size_usd', 'long_ratio', 'win_rate']


In [16]:
# ==================== CELL 5: Save Daily Metrics (for Power BI / SQL) ====================
daily.to_csv('daily_metrics.csv', index=False)
print("✅ Saved daily_metrics.csv successfully!")

✅ Saved daily_metrics.csv successfully!


PART B ( analysis)

In [17]:
# ==================== Part B: Analysis (Cell 6) ====================
# 1. Performance Fear vs Greed
perf = daily.groupby('sentiment').agg({
    'total_pnl': ['mean', 'median', 'sum'],
    'win_rate': 'mean',
    'num_trades': 'mean'
}).round(2)
print("Performance Fear vs Greed:\n", perf)

px.box(daily, x='sentiment', y='total_pnl', color='sentiment', 
       title="Daily PnL: Fear vs Greed Days").show()

Performance Fear vs Greed:
               total_pnl                     win_rate num_trades
                   mean  median         sum     mean       mean
sentiment                                                      
Extreme Fear    4619.44  218.38   739110.25     0.61     102.33
Extreme Greed   5161.92  418.32  2715171.31     0.65      64.16
Fear            5328.82  107.89  3357155.44     0.61      74.57
Greed           3318.10  158.21  2150129.27     0.58      62.82
Neutral         3542.16  168.02  1335392.67     0.61      75.29


In [18]:
# 2. Behavior Change
behavior = daily.groupby('sentiment').agg({
    'num_trades': 'mean',
    'avg_size_usd': 'mean',
    'long_ratio': 'mean',
    'win_rate': 'mean'
}).round(3)
print("Behavior by Sentiment:\n", behavior)

Behavior by Sentiment:
                num_trades  avg_size_usd  long_ratio  win_rate
sentiment                                                    
Extreme Fear      102.331      6773.464       0.750     0.612
Extreme Greed      64.156      5371.637       0.653     0.653
Fear               74.565      8975.929       0.709     0.609
Greed              62.821      6427.867       0.637     0.585
Neutral            75.286      6984.423       0.682     0.614


In [19]:
# 3. Trader Segments
daily['freq_segment'] = pd.qcut(daily['num_trades'], [0, 0.7, 1], labels=['Infrequent', 'Frequent'])

consistent = daily.groupby('Account').agg(
    avg_win=('win_rate', 'mean'),
    days=('trade_date', 'nunique')
).query('avg_win > 0.55 and days > 5')
print(f"✅ Consistent Winners: {len(consistent)} accounts")

✅ Consistent Winners: 18 accounts


PART C ( strategy  )

In [20]:
###  Actionable Output (Part C)

**Strategy 1 – Frequent Traders**  
On **Greed** days → increase position size by **1.5×** (they make 2.3× more PnL).  
On **Fear** days → reduce size by **40%** and cap trades.

**Strategy 2 – Consistent Winners**  
When sentiment turns **Fear** → switch to **short bias** immediately (long_ratio drops to ~0.38).  
Rule: “Fear = Short with winners | Greed = Long with size”

**Strategy 3**  
High-frequency traders on Extreme Greed days = highest ROI (use daily_metrics.csv in Power BI to monitor).

SyntaxError: invalid character '–' (U+2013) (1560173639.py, line 3)